In [1]:
import torch
device = torch.device('cuda')

In [2]:
import torchvision
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_trt.pth'))

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
from jetracer.nvidia_racecar import NvidiaRacecar

robot = NvidiaRacecar()

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [6]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model_trt(x)
    y = F.softmax(y, dim=1)
    
    
    prob_not_road = float(y.flatten()[0])
    
    blocked_slider.value = prob_not_road
    
    time.sleep(0.001)
        
update({'new': camera.value})  

In [7]:
camera.observe(update, names='value')  